In [3]:
import pandas as pd
import numpy as np
from helper_functions import Helper
from helper_functions import CoinAPI
import requests
from typing import List, Dict

In [4]:
def pullAssetInfo(base_url: str, base_headers: Dict[str, str]) -> pd.DataFrame:
    """
    Returns a DataFrame containing information about cryptocurrency assets.
    
    Args:
        base_url: A string representing the base URL of the CoinAPI service.
        base_headers: A dictionary representing the headers to be sent with the API request.

    Returns:
        A Pandas DataFrame containing information about cryptocurrency assets.
    """
    # Build target URL
    target_url = 'assets'
    url        = f"{base_url}{target_url}"
    headers    = base_headers.copy()

    # Call API and convert to DataFrame
    response_json = Helper.makeApiCall(url, headers=headers)
    df = pd.DataFrame(response_json)

    # Convert date columns to datetime
    df['data_start'] = pd.to_datetime(df['data_start'])
    df['data_end'] = pd.to_datetime(df['data_end'])

    # Calculate duration in days
    df['duration_days'] = (df['data_end'] - df['data_start']).dt.days

    # Subset to cryptocurrency assets
    df = df[df['type_is_crypto'] == 1]

    # Subset to assets with trading data
    df = df[~df['data_start'].isnull() & ~df['data_end'].isnull()]

    # Subset to assets with at least four months of history
    df = df[df['duration_days'] > 120]

    # Subset to assets with data start on or before 2022-09-01
    df = df[df['data_start'] <= '2022-09-01']

    return df

In [ ]:
if __name__ == "__main__":
    # import api key
    API_KEY_FP = '../../admin/coinapi.txt'
    with open(API_KEY_FP) as f:
        API_KEY = f.readlines()[0].strip()

    # set args
    PANEL_FP = '../data/raw/coinapi_panel.pkl'
    BASE_URL   = 'https://rest.coinapi.io/v1/'
    BASE_HEADERS = {'X-CoinAPI-Key': API_KEY}
    LEGIT_US_EXCHANGES = ['BINANCEUS', 'BITSTAMP', 'COINBASE', 'CRYPTOCOM', 'FTXUS', 
        'GEMINI', 'KRAKEN', 'KUCOIN']
    TARGET_FREQ = '1DAY'
    TIME_START = '2016-07-01'
    TIME_END = '2023-01-02'

    # confirm api is working
    url = 'https://www.coinapi.io/api/subscriptions/usage/rest/history'
    response = requests.get(url, headers=BASE_HEADERS)
    print(response.json())    

    # pull initial asset universe
    asset_info_df = pullAssetInfo(BASE_URL, BASE_HEADERS)

    # pull relevant markets
    symbols_df = CoinAPI.pullMarketInfo(BASE_URL, BASE_HEADERS, LEGIT_US_EXCHANGES)

    # pull usdt and usdc exchange rates
    prices_df = CoinAPI.pullExchangeRates(BASE_URL, BASE_HEADERS, ['USDC', 'USDT'],
                    TARGET_FREQ, TIME_START, TIME_END)
    macro_df, prices_df = CoinAPI.seperateMacroAndAssetRefPrices(prices_df)

    # pull market prices and volumes
    assert symbols_df.symbol_id.is_unique
    markets_list = symbols_df.symbol_id.values[:10]
    raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                    TARGET_FREQ, TIME_START, TIME_END)
    
    # clean the panel
    panel_df = CoinAPI.cleanPanel(raw_panel_df, None, macro_df, TARGET_FREQ)
    panel_df.to_pickle(PANEL_FP)